In [23]:
import pandas as pd

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
!pip install -q -U datasets
!pip install -q -U bitsandbytes
!pip install -q -U accelerate
!pip install -q -U peft
!pip install -q -U trl
#!pip install -q -U git+https://github.com/huggingface/accelerate.git


In [2]:
pip install -U bitsandbytes

Note: you may need to restart the kernel to use updated packages.


In [24]:
import os
import torch
import transformers
from datasets import load_from_disk
from transformers import (
    BitsAndBytesConfig,
    AutoModelForCausalLM,
    AutoTokenizer,
    Trainer,
    TextStreamer,
    pipeline
)
from peft import (
    LoraConfig,
    prepare_model_for_kbit_training,
    get_peft_model,
    get_peft_model_state_dict,
    set_peft_model_state_dict,
    TaskType,
    PeftModel
)
from trl import SFTTrainer
import pandas as pd
from datasets import load_dataset, Dataset, concatenate_datasets
from transformers import AutoModelForCausalLM, AutoTokenizer

In [25]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch

model_name = "LGAI-EXAONE/EXAONE-3.5-7.8B-Instruct"

quant_config = BitsAndBytesConfig(
    load_in_4bit=True,                   # 4비트 로드 활성화
    bnb_4bit_quant_type="nf4",           # 양자화 방식 (예: "nf4" 또는 "fp4")
    bnb_4bit_use_double_quant=True,      # 이중 양자화 사용 여부
    bnb_4bit_compute_dtype=torch.bfloat16  # 연산 시 사용할 데이터 타입
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    trust_remote_code=True,              # 리포지토리의 custom 코드를 실행하도록 설정
    quantization_config=quant_config,      # BitsAndBytesConfig 객체 전달
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    trust_remote_code=True
)


Loading checkpoint shards: 100%|██████████| 7/7 [00:31<00:00,  4.57s/it]


In [26]:
model

ExaoneForCausalLM(
  (transformer): ExaoneModel(
    (wte): Embedding(102400, 4096, padding_idx=0)
    (drop): Dropout(p=0.0, inplace=False)
    (h): ModuleList(
      (0-31): 32 x ExaoneBlock(
        (ln_1): ExaoneRMSNorm()
        (attn): ExaoneAttention(
          (attention): ExaoneSdpaAttention(
            (rotary): ExaoneRotaryEmbedding()
            (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
            (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
            (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
            (out_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          )
        )
        (ln_2): ExaoneRMSNorm()
        (mlp): ExaoneGatedMLP(
          (c_fc_0): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (c_fc_1): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (c_proj): Linear4bit(in_features=14336, out_features=4096, bias=

In [27]:
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

# Exaone 모델 구조에 맞게 target_modules 목록을 수정
lora_config = LoraConfig(
    r=4,                     # LoRA 가중치 행렬의 rank (값이 작을수록 trainable parameter 수가 줄어듦)
    lora_alpha=8,            # LoRA 스케일링 팩터
    lora_dropout=0.05,
    target_modules=['q_proj', 'k_proj', 'v_proj', 'out_proj', 'c_fc_0', 'c_fc_1', 'c_proj'],
    bias='none',             # bias 파라미터 학습 여부: 'none', 'all', 'lora_only'
    task_type="CAUSAL_LM"     # 문자열로 task_type 지정 (이전에는 TaskType.CAUSAL_LM을 사용했음)
)

# 양자화된 모델 학습 전 전처리
model = prepare_model_for_kbit_training(model)

# PEFT를 사용해 모델에 LoRA 적용
model = get_peft_model(model, lora_config)

# 학습 가능한 파라미터 출력
model.print_trainable_parameters()


trainable params: 10,485,760 || all params: 7,828,934,656 || trainable%: 0.1339


In [28]:
train = pd.read_csv("C:/Users/minkyu/Desktop/open/train.csv", encoding = 'utf-8-sig')

In [29]:
# 데이터 전처리
train['공사종류(대분류)'] = train['공사종류'].str.split(' / ').str[0]
train['공사종류(중분류)'] = train['공사종류'].str.split(' / ').str[1]
train['공종(대분류)'] = train['공종'].str.split(' > ').str[0]
train['공종(중분류)'] = train['공종'].str.split(' > ').str[1]
train['사고객체(대분류)'] = train['사고객체'].str.split(' > ').str[0]
train['사고객체(중분류)'] = train['사고객체'].str.split(' > ').str[1]

In [30]:
# 훈련 데이터 통합 생성
combined_training_data = train.apply(
    lambda row: {
        "question": (
            f"공사종류 대분류 '{row['공사종류(대분류)']}', 중분류 '{row['공사종류(중분류)']}' 공사 중 "
            f"공종 대분류 '{row['공종(대분류)']}', 중분류 '{row['공종(중분류)']}' 작업에서 "
            f"사고객체 '{row['사고객체(대분류)']}'(중분류: '{row['사고객체(중분류)']}')와 관련된 사고가 발생했습니다. "
            f"작업 프로세스는 '{row['작업프로세스']}'이며, 사고 원인은 '{row['사고원인']}'입니다. "
            f"재발 방지 대책 및 향후 조치 계획은 무엇인가요?"
        ),
        "answer": row["재발방지대책 및 향후조치계획"]
    },
    axis=1
)

# DataFrame으로 변환
combined_training_data = pd.DataFrame(list(combined_training_data))

In [31]:
prompt_template = """
### 지침: 당신은 건설 안전 전문가입니다.
질문에 대한 **재발 방지 대책 및 향후 조치 계획**만 간결하게 답변하세요.
- 서론, 배경 설명, 추가 설명 없이 핵심 내용만 전달하세요.
- 불필요한 형식(목차, 강조 표시, 리스트 등)을 사용하지 마세요.
- 한 문장 또는 간결한 문단으로 자연스럽게 작성하세요.
- 특수문자를 포함하지 마세요.

{context}

### 질문:
{question}

### 답변:
"""

In [32]:
from datasets import Dataset

# combined_training_data가 Pandas DataFrame인 경우
dataset_hf = Dataset.from_pandas(combined_training_data.reset_index(drop=True))


In [33]:
def generate_prompt(data_point):
    # 추가 context가 필요한 경우 여기서 설정 (현재는 빈 문자열)
    context = ""
    question = data_point["question"]
    # prompt_template을 사용해 prompt를 생성합니다.
    prompt = prompt_template.format(context=context, question=question)
    # 모델 입력으로 사용할 프롬프트와 정답(평가용)만 반환합니다.
    return {"prompt": prompt, "answer": data_point["answer"]}

# 기존 데이터셋의 모든 컬럼을 제거하고, generate_prompt에서 필요한 컬럼만 남깁니다.
remove_column_keys = list(dataset_hf.features.keys())
dataset_cvted = dataset_hf.shuffle().map(generate_prompt, remove_columns=remove_column_keys)


Map: 100%|██████████| 23422/23422 [00:01<00:00, 14471.78 examples/s]


In [34]:
dataset_cvted

Dataset({
    features: ['answer', 'prompt'],
    num_rows: 23422
})

In [35]:
print(dataset_cvted[0])

{'answer': '타설작업 전 작업방법 및 작업순서를 정하여 근접한 공간에서 타 작업이 동시에 진행되지 않도록 관리하고, 작업지휘자 및 관리감독자, 근로자 안전교육 실시를 통한 유사/동일재해 재발 방지.', 'prompt': "\n### 지침: 당신은 건설 안전 전문가입니다.\n질문에 대한 **재발 방지 대책 및 향후 조치 계획**만 간결하게 답변하세요.\n- 서론, 배경 설명, 추가 설명 없이 핵심 내용만 전달하세요.\n- 불필요한 형식(목차, 강조 표시, 리스트 등)을 사용하지 마세요.\n- 한 문장 또는 간결한 문단으로 자연스럽게 작성하세요.\n- 특수문자를 포함하지 마세요.\n\n\n\n### 질문:\n공사종류 대분류 '건축', 중분류 '건축물' 공사 중 공종 대분류 '건축', 중분류 '가설공사' 작업에서 사고객체 '가시설'(중분류: '비계')와 관련된 사고가 발생했습니다. 작업 프로세스는 '설치작업'이며, 사고 원인은 '콘크리트 타설작업과 인접구간에서 보양용 천막 가설지지물 설치 작업을 동시에 진행, 타설작업구간 접근금지조치 및 통제 미흡'입니다. 재발 방지 대책 및 향후 조치 계획은 무엇인가요?\n\n### 답변:\n"}


In [39]:
def tokenize_function(examples):
    outputs = tokenizer(examples["prompt"], truncation=True, max_length=512)
    return outputs


In [40]:
remove_column_keys = dataset_cvted.features.keys()
dataset_tokenized = dataset_cvted.map(tokenize_function, batched=True, remove_columns=remove_column_keys)

Map: 100%|██████████| 23422/23422 [00:02<00:00, 8796.68 examples/s]


In [44]:
dataset_tokenized

Dataset({
    features: ['input_ids', 'attention_mask'],
    num_rows: 23422
})

In [45]:
def collate_fn(examples):
    examples_batch = tokenizer.pad(examples, padding='longest', return_tensors='pt')
    examples_batch['labels'] = examples_batch['input_ids'] # 모델 학습 평가를 위한 loss 계산을 위해 입력 토큰을 레이블로 사용
    return examples_batch

In [49]:
train_args = transformers.TrainingArguments(
    per_device_train_batch_size=2, # 각 디바이스당 배치 사이즈. 작을수록(1~2) 좀 더 빠르게 alignment 됨
    gradient_accumulation_steps=4,
    warmup_steps=1,
    #num_train_epochs=1,
    max_steps=2000,
    learning_rate=2e-4, # 학습률
    bf16=True, # bf16 사용 (지원되는 하드웨어 확인 필요)
    output_dir="outputs",
    optim="paged_adamw_8bit", # 8비트 AdamW 옵티마이저
    logging_steps=50, # 로깅 빈도
    save_total_limit=3 # 저장할 체크포인트의 최대 수
)

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset_tokenized,
    #max_seq_length=512, # 최대 시퀀스 길이
    args=train_args,
    #dataset_text_field="prompt",
    data_collator=collate_fn
)

Truncating train dataset: 100%|██████████| 23422/23422 [00:05<00:00, 4324.88 examples/s]


In [54]:
model.config.use_cache = False
trainer.train()

AttributeError: `AcceleratorState` object has no attribute `distributed_type`. This happens if `AcceleratorState._reset_state()` was called and an `Accelerator` or `PartialState` was not reinitialized.

In [ ]:
FINETUNED_MODEL = "qlora"
trainer.model.save_pretrained(FINETUNED_MODEL)